In [20]:
import rasterio as rio
import numpy as np
from PIL import Image

In [15]:

# Loading soybean data
soybean_data_path = "../data/raw/yield_soybean.tif"
with rio.open(soybean_data_path) as soybean_src:
    soybean_data = soybean_src.read()
soybean_data = np.squeeze(soybean_data)

In [16]:
def get_lat_long(data, src):
    # index array
    lat_index = np.arange(0, data.shape[0])
    long_index = np.arange(0, data.shape[1])
    
    # meshgrid
    long_grid, lat_grid = np.meshgrid(long_index, lat_index)
    
    # flattened grids
    lat_grid_flat = lat_grid.flatten()
    long_grid_flat = long_grid.flatten()
    
    # getting long and lat
    A = src.transform
    long, lat = rio.transform.xy(A, lat_grid_flat, long_grid_flat)
    
    # reshaping to shape of original data
    lat = np.array(lat).reshape(data.shape)
    long = np.array(long).reshape(data.shape)
    
    return lat, long

def segment_coords(data, lat, long, target_coords = (0, 0), border = 0, length = 10):
    # latitude/longitude variables
    lat_max = target_coords[0]
    lat_min = lat_max - length
    long_min = target_coords[1]
    long_max = long_min + length
    
    # length variables
    xlen = data.shape[1]
    ylen = data.shape[0]
    
    # find xmin and xmax
    xmin, xmax = None, None
    for x in range(xlen):
        if long[0, x] > long_min:
            xmin = x if xmin == None else xmin
        if long[0, xlen - x - 1] < long_max:
            xmax = xlen - x - 1 if xmax == None else xmax
            
    # find ymin and ymax
    ymin, ymax = None, None
    for y in range(ylen):
        if lat[ylen - y - 1, 0] > lat_min:
            ymin = ylen - y - 1 if ymin == None else ymin
        if lat[y, 0] < lat_max:
            ymax = y if ymax == None else ymax
            
    # index data from top-bottom, left-right
    return data[ymax:ymin + border, xmin:xmax + border]

In [17]:
# Segmenting Maize Data 10S60W
target_coords = (-10, -60)
lat, long = get_lat_long(soybean_data, soybean_src)

soybean_data_seg = segment_coords(soybean_data, lat, long, target_coords, border = 1, length = 10)


In [25]:
seg = Image.fromarray(soybean_data_seg)
seg.save("../data/raw/segmented/{lat}{long}/soy_{lat}{long}.tif".format(lat = "10S", long = "60W"))